In [2]:
%pip install psycopg2

## **Creating connection with postgres SQL from python to create and manipulate db and tables**

In [1]:
import psycopg2
import pandas as pd

In [18]:
try:
    conn  = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
except psycopg2.Error as e:
    print("Error: Conuld not connect to postgres db")
    print(e)

In [19]:
try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get the curson to the database")
    print(e)

In [20]:
conn.set_session(autocommit=True)

In [21]:
try:
    cur.execute("CREATE DATABASE pizza_sales_db")
except psycopg2.Error as e:
    print(e)

database "pizza_sales_db" already exists



In [22]:
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

## **Making connection witg out db that was created in postgres SQL**

In [2]:
try:
    conn  = psycopg2.connect("host=127.0.0.1 dbname=pizza_sales_db   user=postgres password=root")
except psycopg2.Error as e:
    print("Error: Conuld not connect to postgres db")
    print(e)

try:
    cur=conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get the curson to the database")
    print(e)

conn.set_session(autocommit=True)

## **As the connection is successfull we are creating all the tables within database**

In [35]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS pizzas(pizza_id int, pizza_type_id varchar, size int, price decimal);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [25]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS pizza_types(pizza_type varchar, name varchar, category varchar, ingredients varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [26]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS orders(order_id int, date date, time time);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

In [27]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS orders_details(order_details_id int,order_id int, pizza_id varchar, quantity int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

## **Importing data from csv to dataframes so that we can insert in to postgres SQL db**

In [36]:
pizzas = pd.read_csv("pizza_sales_data\pizzas.csv")
pizzas.head()

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75


In [22]:
pizza_types = pd.read_csv("pizza_sales_data\pizza_types.csv", encoding='unicode_escape')
pizza_types.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [17]:
orders = pd.read_csv("pizza_sales_data\orders.csv")
orders.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [18]:
order_details = pd.read_csv("pizza_sales_data\order_details.csv")
order_details.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


## **Cleaning the dataset to use it further**

In [26]:
# Checking for duplicates so that we can check the primary keys f each table
dup_pizzaid_pizzas = pizzas['pizza_id'].duplicated()
dup_pizzaid_pizzas_types = pizza_types['pizza_type_id'].duplicated()
dup_orders = orders['order_id'].duplicated()
dup_order_id = order_details['order_details_id'].duplicated()
print("duplicate pizza_id in pizzas: ", dup_pizzaid_pizzas.sum())
print("duplicate pizza_id in pizza_types: ", dup_pizzaid_pizzas_types.sum())
print("duplicate order_id in orders: ", dup_orders.sum())
print("duplicate order_details_id in order_details: ", dup_order_id.sum())

duplicate pizza_id in pizzas:  0
duplicate pizza_id in pizza_types:  0
duplicate order_id in orders:  0
duplicate order_details_id in order_details:  0


In [27]:
#Selecting only the pizzatype_id, name and ingredients as we dont want the category, this will be part of the cleaning
pizza_types_clean = pizza_types[['pizza_type_id','name','ingredients']]
pizza_types_clean.head()

,pizza_type_id,name,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [29]:
print("Null values in pizzas: ", pizzas.isnull().values.any())
print("Null values in pizza_types_clean: ", pizza_types_clean.isnull().values.any())
print("Null values in orders: ", orders.isnull().values.any())
print("Null values in order_details: ", order_details.isnull().values.any())

Null values in pizzas:  False
Null values in pizza_types_clean:  False
Null values in orders:  False
Null values in order_details:  False


## **Inserting table into postgres**

In [76]:
#Need fix
pizzas_table_insert = ("""INSERT INTO pizzas(pizza_id, pizza_type_id, size, price)
                       VALUES(%s, %s, %s, %s)""")

In [ ]:
for i, row in pizzas.iterrows():
    cur.execute(pizzas_table_insert, tuple(row))

In [ ]:
#worked
pizza_types_table_insert = ("""INSERT INTO pizza_types(
                       pizza_type, name, category, ingredients)
                       VALUES(%s, %s, %s,%s)""")

In [ ]:
for i,row in pizza_types.iterrows():
    cur.execute(pizza_types_table_insert,list(row))

In [ ]:
#Order query

In [ ]:
#Order details

In [72]:
#worked
order_details_table_insert = ("""
    INSERT INTO orders_details (order_details_id, order_id, pizza_id, quantity)
    VALUES (%s, %s, %s, %s)""")

In [73]:
for i, row in order_details.iterrows():
    cur.execute(order_details_table_insert, tuple(row))

In [74]:
conn.commit

<function connection.commit>